In [71]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [72]:
def make_batch():
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]
        input_batch.append(np.eye(n_class)[input]) # 独热编码
        target_batch.append(target)
    
    return input_batch, target_batch

In [73]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden) # 7,5
        self.W = nn.Linear(n_hidden, n_class, bias=False) # 5,7
        self.b = nn.Parameter(torch.ones([n_class]))
        
    def forward(self, hidden, X):
        X = X.transpose(0, 1)
        outputs, hidden = self.rnn(X, hidden) # 2,3,7  1,3,5 
        outputs = outputs[-1] # 2,3,5 -> 3,5
        model = self.W(outputs) + self.b # (5,7) * (3,5) -> (3,7)
        return model

In [74]:
n_step = 2
n_hidden = 5 

sentences = ["i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)
batch_size = len(sentences)

In [75]:
model = TextRNN()

In [76]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

input_batch, target_batch = make_batch()
input_batch = torch.FloatTensor(input_batch) # 3,2,7  3为batch_size, 2为sequence_length, 7为字向量长度
target_batch = torch.LongTensor(target_batch) # 3

In [77]:
for epoch in range(5000):
    optimizer.zero_grad()
    
    hidden = torch.zeros(1, batch_size, n_hidden) # 1,3,5
    output = model(hidden, input_batch) 
    loss = criterion(output, target_batch) # (3,7) , (3) -> 1个值
    if (epoch+1) % 1000 == 0:
        print(epoch,': ',loss)
        print(output.shape)
    
    loss.backward()
    optimizer.step()

999 :  tensor(0.1466, grad_fn=<NllLossBackward>)
torch.Size([3, 7])
1999 :  tensor(0.0150, grad_fn=<NllLossBackward>)
torch.Size([3, 7])
2999 :  tensor(0.0053, grad_fn=<NllLossBackward>)
torch.Size([3, 7])
3999 :  tensor(0.0025, grad_fn=<NllLossBackward>)
torch.Size([3, 7])
4999 :  tensor(0.0013, grad_fn=<NllLossBackward>)
torch.Size([3, 7])


### 预测阶段

In [78]:
sentences = ["i love dog", "i love coffee", "i love milk"]
input = [sen.split()[:2] for sen in sentences]
input_batch, target_batch = make_batch()
input_batch = torch.FloatTensor(input_batch)
target_batch = torch.LongTensor(target_batch)
hidden = torch.zeros(1, batch_size, n_hidden)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

[['i', 'love'], ['i', 'love'], ['i', 'love']] -> ['coffee', 'coffee', 'coffee']
